<a href="https://colab.research.google.com/github/haaden/MSDS_498/blob/main/MSDS498_news_press_transcripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import pandas as pd

In [ ]:
import os
import random

import pickle 
import re,string

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
import matplotlib.pyplot as plt

import gensim
from gensim.models import Word2Vec,LdaMulticore, TfidfModel, CoherenceModel
from gensim import corpora
from gensim import similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import corpora
from gensim.models import LsiModel,LdaModel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_doc(doc): 
    #split document into individual words
    tokens=doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 4]
    #lowercase all words
    tokens = [word.lower() for word in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]         
    # word stemming    
    # ps=PorterStemmer()
    # tokens=[ps.stem(word) for word in tokens]
    return tokens

def final_processed_text(doc):
    #this is a function to join the processed text back
    ' '.join(doc)
    return doc

download data


In [ ]:


from urllib.request import urlopen, Request

import requests

# To extract fundamental data
from bs4 import BeautifulSoup

demo= '97591392fd6d2255da9db4d2b086bac5'



In [ ]:
sp_500= requests.get(f'https://financialmodelingprep.com/api/v3/sp500_constituent?apikey={demo}').json()

In [ ]:
sp_500df= pd.DataFrame(sp_500)

In [ ]:
#sp_500df_energy = sp_500df[sp_500df['sector']=='Energy']

In [ ]:
#sp_500_energy_lst= sp_500df[sp_500df['sector']=='Energy']['symbol'].tolist()

In [ ]:
sp_500_tickers= sp_500df['symbol'].tolist()

In [ ]:
no_data = []
transcripts_all= []

for company in sp_500_tickers:
  transcript = requests.get(f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{company}?quarter=1&year=2023&apikey={demo}').json()
  if transcript:
    try:
      transcript = pd.DataFrame(transcript)
      transcripts_all.append(transcript)
    except:
      next
#press= requests.get(f"https://financialmodelingprep.com/api/v3/press-releases/{company}?&apikey={demo}").json()
#news= requests.get(f"https://financialmodelingprep.com/api/v3/stock_news?{company}?&apikey={demo}").json()



In [ ]:
transcriptdf= pd.concat(transcripts_all)

In [ ]:
transcriptdf.head(1)

In [ ]:
press_release = []
for company in sp_500_tickers:
  press = requests.get(f"https://financialmodelingprep.com/api/v3/press-releases/{company}?&apikey={demo}").json()
  if press:
    try:
      press = pd.DataFrame(press)
      press_release.append(press )
    except:
      next


In [ ]:
pressdf= pd.concat(press_release)

In [ ]:
news = []
for company in sp_500_tickers:
  nw = requests.get(f"https://financialmodelingprep.com/api/v3/stock_news?tickers={company}&limit=50?&apikey={demo}").json()

  if nw:
    try:
      nw = pd.DataFrame(nw)
      news.append(nw )
    except:
      next

In [ ]:
newsdf= pd.concat(news)


In [ ]:
newsdf.shape

(25037, 7)

In [ ]:
transcriptdf.to_pickle("transcriptdf_sp500.pkl")
newsdf.to_pickle('newsdf_sp500.pkl')
pressdf.to_pickle('pressdf_sp500.pkl')

In [ ]:
nw = newsdf.copy()

In [ ]:
nw.columns

Index(['symbol', 'publishedDate', 'title', 'image', 'site', 'text', 'url'], dtype='object')

In [ ]:
nw = nw.drop(['image','url'], axis=1)

In [ ]:
nw= nw[nw['publishedDate']>='2022-12-31']

In [ ]:
pressdf= pressdf.copy()

In [ ]:
pressdf= pressdf[pressdf['date']>'2022-12-31']

In [ ]:
transdf = transcriptdf.copy()

In [ ]:
transdf.columns

Index(['symbol', 'quarter', 'year', 'date', 'content'], dtype='object')

In [ ]:
#data= data[data["Movie Title"]=="Frozen_2"]
#adding two columns to the dataframe to store the processed text and tokenized text
nw['processed_text'] = nw['text'].apply(lambda x: clean_doc(x))

#creating final processed text variables for matrix creation
nw['final_processed_text'] = [' '.join(x) for x in nw['processed_text']]

In [ ]:
len(pressdf['text'].tolist())

7353

In [ ]:
#data= data[data["Movie Title"]=="Frozen_2"]
#adding two columns to the dataframe to store the processed text and tokenized text
pressdf['processed_text'] = pressdf['text'].apply(lambda x: clean_doc(x))

#creating final processed text variables for matrix creation
pressdf['final_processed_text'] = [' '.join(x) for x in pressdf['processed_text']]

In [ ]:
# cleaning transcripts

In [ ]:
translist = transdf.content.str.split("\n").tolist()

In [ ]:
transdf.head(1)

,symbol,quarter,year,date,content
0,MMM,1,2023,2023-04-25 14:20:17,"Operator: Ladies and gentlemen, thank you for ..."


In [ ]:
#transdf['text_clean'].apply(lambda x: [''.join(a) for a in x]).tolist()

In [ ]:
#transdf['final_processed_text']= [' '.join(x) for x in transdf['text_clean']]

In [ ]:
#transdf= transdf.drop('final_processed_text',axis=1)

In [ ]:
clean_text= []
for i in translist:
  txt= []
  for a in i:
    doc= clean_doc(a)
    if len(doc)>512:
      docsplit = a.split(".")
      #print(docsplit)
      for c in docsplit:
        #print(c)
        if len(c)>512:
           doc_split= c.split(",")
           for d in doc_split:
              doc = clean_doc(d)
              txt.append(doc)
        else:    
            doc= clean_doc(c)
            txt.append(doc)
    else:
      txt.append(doc)
    
  clean_text.append(txt)

In [ ]:
final_text=[]
for i in range(len(clean_text)):
  txt=[]
  for a in range(len(clean_text[i])):
    #print(clean_text[i])
    txt.append([' '.join(clean_text[i][a])])  #(x) for x in clean_text[i]])
  final_text.append(txt)

In [ ]:
len(final_text)

481

In [ ]:
for i in translist[22]:
  print(len(i))
  if len(i)>512:
    print(i.split("."))

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

In [ ]:
# tested in transformers==4.18.0 
import transformers
transformers.__version__

'4.29.2'

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg',truncation=True, max_length=512)

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer, truncation=True, max_length=512)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
pip install xformers

In [ ]:
# only Q1 news
#nw= nw[nw['publishedDate']>'2022-12-31'].copy()

In [ ]:
nw.shape

(1048, 7)

In [ ]:
nw["ESG"]= nw['final_processed_text'].apply(nlp)

In [ ]:
ls= nw['ESG'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
ls.head(1)

,label,score
0,None,0.93525


In [ ]:
nw = pd.concat((nw,ls ), axis=1 )

In [ ]:
nw.to_pickle("nw_ESG_ps500.pkl")

In [ ]:
from google.colab import files
files.download("nw_ESG_ps500.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#pressdf

Index(['symbol', 'date', 'title', 'text'], dtype='object')

In [ ]:
pressdf["ESG"]= pressdf['final_processed_text'].apply(nlp)

In [ ]:
pls= pressdf['ESG'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
pressdf= pd.concat((pressdf,pls ), axis=1 )

In [ ]:
pressdf.to_pickle('pressESG_sp500.pkl')

In [ ]:
from google.colab import files
 
files.download('pressESG_sp500.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
final_text[22][3]

In [ ]:
ESG=[]

for i in range(len(clean_text)):
  esg=[]
  for a in clean_text[i]:
    print(len(a))
    if len(a)<=512:
      doc = ' '.join(a)
      esg.append(nlp(doc))
    else:
      a= a[0:512]
      doc = ' '.join(a)
      esg.append(nlp(doc))
  ESG.append(esg)


In [ ]:
pd.DataFrame(ESG).T.to_pickle("TransESG.pkl")

In [ ]:
pd.DataFrame(final_text).T.to_pickle("final_text.pkl")

In [ ]:
files.download("TransESG.pkl")
files.download("final_text.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for i in clean_text:
  for a in i:
    if len(a)==488:
      doc= ' '.join(a)
      print(doc)
      #nlp(doc)

thank morning welcome first quarter conference today discuss continued progress executing strategy cover operations rielly review financial results believe offers unique value proposition investors strategy deliver highreturn resource growth supply industryleading growth maintain industry leadership environmental social governance performance disclosure terms resource growth multiple phases guyana developments coming online robust inventory highreturn drilling locations bakken deliver highly profitable production growth annually stabroek block guyana currently sight floating production storage floating vessels fpsos gross production capacity million barrels terms supply resource continues expand steadily curve forecast costs decline approximately portfolio achieve breakeven brent price approximately barrel sanctioned developments stabroek block breakeven brent price approximately barrel terms growth industryleading change story industryleading duration story providing highly differenti

In [ ]:
t= 'thank morning welcome first quarter conference today discuss continued progress executing strategy cover operations rielly review financial results believe offers unique value proposition investors strategy deliver highreturn resource growth supply industryleading growth maintain industry leadership environmental social governance performance disclosure terms resource growth multiple phases guyana developments coming online robust inventory highreturn drilling locations bakken deliver highly profitable production growth annually stabroek block guyana currently sight floating production storage floating vessels fpsos gross production capacity million barrels terms supply resource continues expand steadily curve forecast costs decline approximately portfolio achieve breakeven brent price approximately barrel sanctioned developments stabroek block breakeven brent price approximately barrel terms growth industryleading change story industryleading duration story providing highly differentiated value proposition based brent price barrel barrel forecast increase approximately annually twice topline growth balance sheet continue strengthen recent debttoebitdax ratio approximately onetime successful execution strategy uniquely positioned company deliver significant value shareholders years growing intrinsic value growing returns financial priorities allocate capital highreturn lowcost investment opportunities maintain strong balance sheet position ensure worldclass investment opportunities guyana bakken allocated capital budget return annual shareholders dividend increases share repurchases return capital framework march increased annual dividend share looking ahead continue increasing regular dividend level attractive income oriented investors sustainable price environment generation steadily increases future years share repurchases expected represent growing proportion return capital manage price volatility hedged barrels barrels barrel options barrels barrel brent options positions shareholders protected downside fully benefiting upside strategy guyana industrys largest province discovered decade interest exxonmobil operator since discoveries block including since start fangtooth underpinning gross discovered recoverable resource estimate billion barrels equivalent multibillion barrels exploration potential remaining potential fpsos develop discovered resources block phase phase developments produced average approximately gross barrels first quarter third sanctioned development payara arrived stabroek block earlier month ahead schedule targeted start early fourth quarter gross production capacity approximately fourth sanction development yellowtail expected online gross production capacity approximately barrels government regulatory approvals expected hopefully fifth development gross production capacity approximately barrels development sixth development whiptail expected submitted government regulatory government approvals later turning bakken continue operating fourrig program enable production approximately barrels equivalent lower fully optimize infrastructure generate significant levels continue outstanding applying manufacturing principles build culture innovation improve efficiency mitigate inflationary pressures execute company strategy continue guided longstanding commitment sustainability proud industry leader earlier month announced million donation years institute harnessing plants initiative potential changer tackling global challenge climate change developing plants crops wetlands natural ability capture store potentially billions carbon atmosphere proud received rating latest environmental social governance rating assessment mscis highest rating designates company leader managing industryspecific risks relative peers received first rating earning ratings consecutive years february earned place bloomberg gender equality index fourth consecutive summary continue successfully execute strategy offers unique value proposition industry growing intrinsic value returns multiple phases lowcost developments coming online guyana robust inventory highreturn drilling locations bakken portfolio positioned become increasingly positive continue prioritize return capital shareholders dividend increases share repurchases operational update'

In [ ]:
transdf

In [ ]:
tESG= pd.DataFrame(ESG).T#.columns= transdf['symbol'].tolist()

In [ ]:
tESG.columns=transdf['symbol'].tolist()

In [ ]:
tESG=tESG.T.reset_index(names='symbol')

In [ ]:
#tESG= tESG.stack()#.reset_index()#.drop('level_0', axis=1)

In [ ]:
tESG= pd.melt(tESG, id_vars=["symbol"], 
                   value_name="Value")

In [ ]:
tESG.columns = ['symbol', 'variable', 'ESG']

In [ ]:
tESG = tESG.dropna()

In [ ]:
tls= tESG['ESG'].apply(lambda x: x[0]).apply(pd.Series)

In [ ]:
tESGdf= pd.concat((tESG,tls),axis=1) 

In [ ]:
tESGdf.columns= ['symbol', 'number', 'ESG', 'label', 'score']

In [ ]:
ftxt= pd.DataFrame(final_text).T

In [ ]:
ftxt.columns=transdf['symbol'].tolist()

In [ ]:
ftxt= ftxt.T.reset_index(names='symbol')

In [ ]:
ftxt= pd.melt(ftxt, id_vars=["symbol"], 
                   value_name="Value")

In [ ]:
ftxt.columns= ['symbol',"number","final_text"]

In [ ]:
ftxt= ftxt.dropna()

In [ ]:
ftxt.shape

(91532, 3)

In [ ]:
tESGdf.shape

(91532, 5)

In [ ]:
transtxtESG= ftxt.merge(tESGdf, on=['symbol','number'])

In [ ]:
transESGdf= transdf.merge(transtxtESG, on='symbol')

In [ ]:
transESGdf.to_pickle("transESGdf_sp500.pkl")

In [ ]:
files.download("transESGdf_sp500.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
transESGdf

In [ ]:
TransESGscore= transESGdf[(transESGdf['score']>0.8) & (transESGdf['label']!='None') ].copy()

In [ ]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',truncation=True, max_length=512)

In [ ]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer,truncation=True, max_length=512)

In [ ]:
TransESGscore["ESG_SCORE"]=TransESGscore["final_text"].apply(nlp)

In [ ]:
tes= TransESGscore["ESG_SCORE"].apply(lambda x: pd.Series(x[0]))

In [ ]:
tes.columns= ['label_s','score_s']

In [ ]:
TransESGfinal= pd.concat((TransESGscore,tes),axis=1)

In [ ]:
TransESGfinal.to_pickle("final_trans_500.pkl")

In [ ]:
files.download("transESGdf_sp500.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
nwESG= pd.read_pickle("nw_ESG_ps500.pkl")

In [ ]:
nwESGdf= nwESG[(nwESG['label']!='None')&(nwESG['score']>0.8)].copy()

In [ ]:
nwESGdf['ESG_score']=nwESGdf['final_processed_text'].apply(nlp)

In [ ]:
nes = nwESGdf['ESG_score'].apply(lambda x: pd.Series(x[0]))

In [ ]:
nes.columns= ['label_s','score_s']

In [ ]:
nwESGfinal= pd.concat((nwESGdf, nes),axis=1)

In [ ]:
nwESGfinal.to_pickle('nwESGfinal_sp500.pkl')

In [ ]:
files.download('nwESGfinal_sp500.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
nwESGfinal.head(1)

,symbol,publishedDate,title,site,text,processed_text,final_processed_text,ESG,label,score,ESG_score,label_s,score_s
11,APA,2023-03-24 07:56:00,Midstream Caught In Middle Of Permian Producer...,Seeking Alpha,Public and private producers are parting ways ...,"[public, private, producers, parting, permian,...",public private producers parting permian basin...,"[{'label': 'Social', 'score': 0.86109918355941...",Social,0.861099,"[{'label': 'Neutral', 'score': 0.9998087286949...",Neutral,0.999809


In [ ]:
pressESG= pd.read_pickle("pressESG_sp500.pkl")

In [ ]:
pressESG.head(1)

,symbol,date,title,text,processed_text,final_processed_text,ESG,label,score
0,APA,2023-05-03 16:15:00,APA CORPORATION ANNOUNCES FIRST-QUARTER 2023 F...,"HOUSTON, MAY 03, 2023 (GLOBE NEWSWIRE) -- APA ...","[houston, globe, newswire, corporation, nasdaq...",houston globe newswire corporation nasdaq toda...,"[{'label': 'None', 'score': 0.9842051863670349}]",None,0.984205


In [ ]:
pressESGdf = pressESG[(pressESG['label']!='None')&(pressESG['score']>0.8)].copy()

In [ ]:
pressESGdf['ESG_score']=pressESGdf['final_processed_text'].apply(nlp)

In [ ]:
pes = pressESGdf['ESG_score'].apply(lambda x: pd.Series(x[0]))

In [ ]:
pes.columns= ['label_s','score_s']

In [ ]:
pressESGfinal= pd.concat((pressESGdf, pes),axis=1)

In [ ]:
pressESGfinal.head(1)

,symbol,date,title,text,processed_text,final_processed_text,ESG,label,score,ESG_score,label_s,score_s
1,APA,2023-03-01 17:05:00,APA CORPORATION ANNOUNCES 2023 ESG GOALS,"HOUSTON, MARCH 01, 2023 (GLOBE NEWSWIRE) -- AP...","[houston, march, globe, newswire, corporation,...",houston march globe newswire corporation nasda...,"[{'label': 'Environmental', 'score': 0.9437037...",Environmental,0.943704,"[{'label': 'Positive', 'score': 0.703463196754...",Positive,0.703463


In [ ]:
pressESGfinal.to_pickle('pressESGfinal_sp500.pkl')

In [ ]:
files.download('pressESGfinal_sp500.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
TransESGfinal.head(1)

,symbol,quarter,year,date,content,textsp,text_clean,number,final_text,ESG,label,score,ESG_SCORE,label_s,score_s
14,APA,1,2023,2023-05-04 15:41:06,"Operator: Good day, and thank you for standing...","[Operator: Good day, and thank you for standin...","[operator, thank, standing, welcome, conferenc...",14,[christmann egypt couple minutes country almos...,"[{'label': 'Social', 'score': 0.94956123828887...",Social,0.949561,"[{'label': 'Positive', 'score': 0.999994874000...",Positive,0.999995


In [ ]:
TransESGfinal_score= TransESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
TransESGfinal_score.columns=  ['label_count','senitiment_count', 'score_s']

In [ ]:
pressESGfinal_score= pressESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
pressESGfinal_score.columns= ['label_count','senitiment_count', 'score_s']

In [ ]:
pressESGfinal_score= pressESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
pressESGfinal_score.columns = ['label_count','senitiment_count', 'score_s']

In [ ]:
nwESGfinal_score = nwESGfinal.groupby(['symbol','label','label_s']).agg({"label":'count',"label_s":'count','score_s': 'mean'})

In [ ]:
nwESGfinal_score.columns= ['label_count','senitiment_count', 'score_s']

In [ ]:
pressESGfinal_score.loc[['COP','XOM']]

label_count  senitiment_count   score_s
symbol label         label_s                                          
COP    Environmental Positive            1                 1  0.999067
       Social        Neutral             3                 3  0.863043
XOM    Environmental Neutral            19                19  0.995695
                     Positive            5                 5  0.988097
       Governance    Neutral             3                 3  0.999911
                     Positive            1                 1  0.999260
       Social        Negative            1                 1  0.742412
                     Neutral            12                12  0.982514
                     Positive            3                 3  0.939571

In [ ]:
nwESGfinal_score.loc[['COP','XOM']]

label_count  senitiment_count   score_s
symbol label         label_s                                          
COP    Social        Negative            1                 1  0.978692
                     Neutral            10                10  0.967055
                     Positive            3                 3  0.989896
XOM    Environmental Negative            2                 2  0.818651
                     Neutral             2                 2  0.960986
                     Positive            1                 1  0.999998
       Social        Negative            2                 2  0.966253
                     Neutral             6                 6  0.996266

In [ ]:
TransESGfinal_score.loc[['XOM']]

label_count  senitiment_count   score_s
symbol label  label_s                                          
XOM    Social Neutral             1                 1  0.713562
              Positive            2                 2  0.997577